<a href="https://colab.research.google.com/github/shiyuhu1933/EC-523-final-project/blob/main/SE-block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import random
import math
import datetime
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.utils import data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision import transforms as T
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
from torchvision.utils import save_image
from torch.backends import cudnn

import numpy as np 
from PIL import Image
from tqdm import tqdm 
import random 

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch import nn, optim

In [6]:
class SE_block(nn.Module):
  def __init__(self, ch, kn, stride, padding, enable):
    super(SE_block, self).__init__()
    self.ch, self.kn, self.stride, self.padding, self.enable  = ch, kn, stride, padding, enable
    self.conv1 = nn.Conv2d(ch, ch, kn, stride, padding, bias=True)
    self.conv1_norm = nn.InstanceNorm2d(ch)
    self.conv2 = nn.Conv2d(ch, ch, kn, stride, padding, bias=True)
    self.conv2_norm = nn.InstanceNorm2d(ch)

    self.se_conv1 = nn.Conv2d(ch, ch//16, kernel_size = 1)
    self.se_conv2 = nn.Conv2d(ch//16, ch, kernel_size = 1)
  def forward(self, x):
    output = F.relu(self.conv1_norm(self.conv1(x)))
    output = self.conv2_norm(self.conv2(output))
    if self.enable:
      se = F.avg_pool2d(output, output.size(2))
      se = F.relu(self.se_conv1(se))
      se = F.sigmoid(self.se_conv2(se))
      output = se * output
    output = output + x
    output = F.relu(output)
    return output 
